In [8]:
import torch
import numpy as np

In [14]:
a = torch.zeros((3,))
print(f"1w维矩阵 在 torch 中表示：{a.shape}")
a = np.zeros((3,)) 
print(f"1w维矩阵 在 numpy 中表示：{a.shape}")

1w维矩阵 在 torch 中表示：torch.Size([3])
1w维矩阵 在 numpy 中表示：(3,)


In [24]:
a = torch.zeros((3,))
a = a.type(torch.int)

In [70]:
x = torch.ones(2, 2, requires_grad=True)
x

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)

In [109]:
y = x**2
y

tensor([[1., 1.],
        [1., 1.]], grad_fn=<PowBackward0>)

### grad_fn记录了y由x计算的过程 
叶子节点无grad_fn, 如x, 由x计算得到的有grad_fn
PowBackward0 用power的求导法则

In [75]:
## 对y=x^2 在x=1处求导 2x
y.grad_fn 

### grad：当执行完了backward()之后，通过x.grad查看x的梯度值。

In [61]:
out = y.mean()
out.backward(torch.tensor(2.))
print(x.grad)

tensor([[1., 1.],
        [1., 1.]])


In [48]:
out

tensor(1., grad_fn=<MeanBackward0>)

In [57]:
print(y.backward(torch.ones_like(y)))

None


In [103]:
import torch

a = torch.tensor([2., 3.], requires_grad=True)
b = torch.tensor([6., 4.], requires_grad=True)
Q = 3*a**3 - b**2

#### 此时Q为vector, 如果要使用Q.backward(); 需要指定与Q同样维度的输入，否则中间求导过程无法进行
如果为标量，则可以直接计算梯度;所以模型中定义loss时，结果都是标量

In [100]:
external_grad = torch.tensor([1., 1.])
Q.backward(gradient=external_grad)
a.grad

tensor([36., 81.])

In [108]:
## grad累加的
Q2 = 3*a**3 - b**2
Q2.sum().backward() #
a.grad

tensor([144., 324.])

In [101]:
b.grad

tensor([-12.,  -8.])

In [104]:
Q.sum().backward() ##与前面结果一样

In [89]:
b.grad

tensor([-12.,  -8.])

#### RuntimeError: Trying to backward through the graph a second time, but the saved intermediate results have already been freed. Specify retain_graph=True when calling .backward() or autograd.grad() the first time.

针对这样的错误，是因为backward一次时，计算图中的中间变量在计算完后就会被释放， 再次使用相关数据会报错

### 在模型训练过程中，计算完loss后使用backward(), 根据计算图保存所有参数的梯度
1、Autograd then calculates and stores the gradients for each model parameter in the parameter’s .grad attribute

2、然后通过优化器启动梯度下降，根据变量的grad更新梯度    

部分参考：

https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html#